In [401]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import pandas as pd
from pandas import ExcelWriter
import spacy
from elasticsearch import helpers, Elasticsearch
import csv
import nltk
import numpy as np
from collections import Counter
import string
import re
nltk.download('stopwords')
from textblob import Word
from itertools import islice
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Shree\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [402]:
nlp = spacy.load("en_core_web_sm")

In [404]:
#Define files
filepath = os.getcwd()
filename1 = filepath + '\RawSurveyData2019.csv'
filename2 = filepath + '\SurveyData2016-2018.csv'
#print(filename1)

In [405]:
#read in csv's as a dataframe using pandas
newdata = pd.read_csv(filename1)
#olddata = pd.read_csv(filename2)

In [407]:
#view data
#newdata.head()


In [408]:
#view data
#olddata.head()

In [409]:
#define stop words and columns for newdata
newdata = pd.DataFrame(newdata)

#rename columns
newdata.columns = ["date_opened","category","cc_rating","resolved", "contact_customer", "cc_comments","eva_rating", "eva_comments", "cc_specialist"]


In [410]:
def sentiment_calc(text):
    try:
       # print(text)
        return round(TextBlob(text).sentiment.polarity, 2)
    except:
        return None   


In [411]:
def subjectivity_calc(text):
    try:
       # print(text)
        return round(TextBlob(text).sentiment.subjectivity, 2)
    except:
        return None

In [412]:
def preprocessing(text):
    try:
        # Tokenize into words
        tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    
        # Remove stopwords
        stop = stopwords.words('english')
        tokens = [token for token in tokens if token not in stop]
        
        # Remove punctuation
        punc = set(string.punctuation)
        tokens = [token for token in tokens if token not in punc]
        
        # lower case
        tokens = [word.lower() for word in tokens]
        
        # lemmatize
        #lmtzr = WordNetLemmatizer()
        #tokens = [lmtzr.lemmatize(word) for word in tokens]
        preprocessed_text= ' '.join(tokens)
    
        return preprocessed_text 
    except:
        return text

In [413]:
# Replace [No Answer Entered] with NaN
newdata = newdata.replace({'[No Answer Entered]': np.nan})

In [414]:
# Perform tokenization, stopword, remove punctuation and lower case conversion for cc_comments 
for column in newdata[['cc_comments']]:
    columnValue = newdata[column]
    newdata[column+'_polarity_score'] = newdata[column].apply(sentiment_calc)
    newdata[column+'_subjectivity_score'] = newdata[column].apply(subjectivity_calc)
    newdata[column+'_processed'] = newdata[column].apply(preprocessing)

# Perform tokenization, stopword, remove punctuation and Lower case conversion for eva_comments 
for column in newdata[['eva_comments']]:
    newdata[column+'_polarity_score'] = newdata[column].apply(sentiment_calc)
    newdata[column+'_subjectivity_score'] = newdata[column].apply(subjectivity_calc)
    newdata[column+'_processed'] = newdata[column].apply(preprocessing)
 

In [415]:
#newdata.head()

### Note on above code: we can visit stemming if needed because I found this was an issue with the other dataset.

### Assign new columns with interpretations of the sentiment and polarity scores. This was helpful to make charts in Kibana

In [416]:
#on cc_comments assign positive, negative, or neutral to polarity in a new column
newdata.loc[newdata.cc_comments_polarity_score < 0, 'cc_sentiment'] = 'negative' 
newdata.loc[newdata.cc_comments_polarity_score == 0, 'cc_sentiment'] = 'neutral' 
newdata.loc[newdata.cc_comments_polarity_score > 0, 'cc_sentiment'] = 'positive'

#on eva_comments assign positive, negative, or neutral to polarity in a new column. 
newdata.loc[newdata.eva_comments_polarity_score < 0, 'eva_sentiment'] = 'negative' 
newdata.loc[newdata.eva_comments_polarity_score == 0, 'eva_sentiment'] = 'neutral' 
newdata.loc[newdata.eva_comments_polarity_score > 0, 'eva_sentiment'] = 'positive'

In [417]:
#on cc_comments assign objective or subjective to subjectivity in a new column based on midpoint between 0-1.
newdata.loc[newdata.cc_comments_subjectivity_score <= 0.5, 'cc_subjectivity'] = 'objective' 
newdata.loc[newdata.cc_comments_subjectivity_score > 0.5, 'cc_subjectivity'] = 'subjective'

#on eva_comments assign objective or subjective to subjectivity in a new column based on midpoint between 0-1.
newdata.loc[newdata.eva_comments_subjectivity_score <= 0.5, 'eva_subjectivity'] = 'objective' 
newdata.loc[newdata.eva_comments_subjectivity_score > 0.5, 'eva_subjectivity'] = 'subjective'

### Do we need to subset each question to remove NaN's? It would make a cleaner set to analyze. I've done that to create a dataset for each question.

In [399]:
newdata.head()

,date_opened,category,cc_rating,resolved,contact_customer,cc_comments,eva_rating,eva_comments,cc_specialist,cc_comments_polarity_score,cc_comments_subjectivity_score,cc_comments_processed,eva_comments_polarity_score,eva_comments_subjectivity_score,eva_comments_processed,cc_sentiment,eva_sentiment,cc_subjectivity,eva_subjectivity
0,1/24/2019,Vendor,5,Yes,No,It was very quick and easy.,5,NaN,"Jennifer, Specialist",0.43,0.74,it quick easy,NaN,NaN,NaN,positive,NaN,subjective,NaN
1,1/24/2019,Buyer,5,Yes,No,The customer care rep. was very helpful and ve...,5,very satisfied with my experience with the eVA...,"Devyonne, Specialist",0.20,0.30,the customer care rep. helpful patient requests,0.65,1.0,satisfied experience eva rep,positive,positive,objective,subjective
2,1/24/2019,Buyer,5,No,No,We had EP and VDC said it was in composing and...,5,NaN,"Portia, Specialist",0.50,0.50,we ep vdc said composing received i told gone ...,NaN,NaN,NaN,positive,NaN,objective,NaN
3,1/24/2019,Vendor,5,Yes,No,NaN,5,NaN,"Miranda, Specialist",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1/24/2019,Vendor,5,Yes,No,Jennifer was very helpful and resolved my ques...,5,I just contacted the customer care phone number,"Jennifer, Specialist",0.20,0.30,jennifer helpful resolved question,0.00,0.0,i contacted customer care phone number,positive,neutral,objective,objective


In [400]:
#select columns and call it "newdata_cc"
newdata_cc = newdata[['date_opened','category', 'resolved','cc_rating', 'cc_comments', 'cc_comments_polarity_score', 'cc_comments_sentiment_score', 'cc_sentiment', 'cc_subjectivity']]

KeyError: "['cc_comments_sentiment_score'] not in index"

In [ ]:
#check subset
newdata_cc.head()

In [ ]:
#drop na rows for newdata_cc_comments
newdata_cc = newdata_cc.dropna()

newdata_cc.head()

### repeat subset for second survey question eva_comments

In [ ]:
#select columns and call it newdata_eva
newdata_eva = newdata[['date_opened','category', 'resolved','eva_rating', 'eva_comments', 'eva_comments_polarity_score', 'eva_comments_sentiment_score', 'eva_sentiment', 'eva_subjectivity']]

In [ ]:
#check subset
newdata_eva.head()

In [ ]:
#drop na rows for newdata_cc_comments
newdata_eva = newdata_eva.dropna()

newdata_eva.head()

In [ ]:
#export both question sets as a csv for elasticsearch
newdata_cc.to_csv(r'C://Users//annac//Desktop//DAPT//Text Mining//newdata_cc.csv', index = False)
newdata_eva.to_csv(r'C://Users//annac//Desktop//DAPT//Text Mining//newdata_eva.csv', index = False)

In [ ]:
#Use this indexing in Kibana. This must be entered into Kibana under the "wrench" tool. 
#NOT CODE TO RUN IN PYTHON

PUT /q1
{
   "settings": {
    "analysis": {
      "analyzer": {
        "std_english": { 
          "type":      "standard",
          "stopwords": "_english_"
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "resp_id":    { "type": "keyword" },  
      "q1":  { "type": "text" ,"analyzer": "standard", "fielddata": true }, 
      "q1_polarity_score":   { "type": "integer"  },     
      "q1_subjectivity_score":   { "type": "integer"  },
      "sentiment":    { "type": "keyword" },
      "subjectivity":    { "type": "keyword" }
    }
  }
}

#Once you enter this in Kibana, THEN you can run the below code to push the data in and it will index properly.

In [ ]:
#Code used to push to elasticsearch (after index mapping above is set in Kibana)
from elasticsearch import helpers, Elasticsearch
import csv

In [ ]:
es = Elasticsearch()

In [ ]:
with open('q1.csv') as f:
    reader = csv.DictReader(f)
    helpers.bulk(es, reader, index='q1')

### We can repeat this code if we want to for the "olddata" file we imported which is from 2016-2018. NOTE: this dataset only has 1 survey question on Customer Care comments (no eva comments- that was added in 2019)

#### Once we finalize the code and all of us get the data into kibana, we can make some visualizations and screen shot them. I can show you that. 